In [1]:
import pennylane as qml
import random
import os
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
date = '20211014_151730'

resume_iters = 120

test_sample_size = 5000

layer =1

z_dim = 8

In [4]:
model_dir_path = r'/home/ken/projects/QuantumGAN-PyTorch/results/GAN/'+date+'/train/model_dir'

In [5]:
def sample_z(batch_size):
    return np.random.normal(0, 1, size=(batch_size, z_dim))

In [6]:
z = sample_z(test_sample_size)

# Data

In [7]:
from data.sparse_molecular_dataset import SparseMolecularDataset

In [8]:
mol_data_dir = 'data/gdb9_9nodes.sparsedataset'

In [9]:
data = SparseMolecularDataset()

data.load(mol_data_dir)

# Generator

In [10]:
from models.models import Generator, Discriminator

In [11]:
complexity = 'nr'

dropout = 0.0

In [12]:
if complexity == 'nr':
    g_conv_dim = [128, 256, 512]
elif complexity == 'mr':
    g_conv_dim = [128]
elif complexity == 'hr':
    g_conv_dim = [16]
else:
    raise ValueError("Please enter an valid model complexity from 'mr', 'hr' or 'nr'!")

In [13]:
G = Generator(g_conv_dim, z_dim, data.vertexes, data.bond_num_types, data.atom_num_types, dropout)

In [14]:
G.to(device)

Generator(
  (activation_f): Tanh()
  (multi_dense_layers): MultiDenseLayers(
    (activation): Tanh()
    (linear_layers): Sequential(
      (0): Linear(in_features=8, out_features=128, bias=True)
      (1): Tanh()
      (2): Dropout(p=0.0, inplace=False)
      (3): Linear(in_features=128, out_features=256, bias=True)
      (4): Tanh()
      (5): Dropout(p=0.0, inplace=False)
      (6): Linear(in_features=256, out_features=512, bias=True)
      (7): Tanh()
      (8): Dropout(p=0.0, inplace=False)
    )
  )
  (edges_layer): Linear(in_features=512, out_features=405, bias=True)
  (nodes_layer): Linear(in_features=512, out_features=45, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)

In [15]:
G_path = os.path.join(model_dir_path, '{}-G.ckpt'.format(resume_iters))

In [16]:
G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))

<All keys matched successfully>

# Generating Molecules

In [17]:
import torch.nn.functional as F

In [18]:
z = torch.from_numpy(z).to(device).float()

In [19]:
post_method = 'softmax'

In [20]:
edges_logits, nodes_logits = G(z)

In [21]:
def postprocess(inputs, method, temperature=1.0):
    def listify(x):
        return x if type(x) == list or type(x) == tuple else [x]
    def delistify(x):
        return x if len(x) > 1 else x[0]
    if method == 'soft_gumbel':
        softmax = [F.gumbel_softmax(e_logits.contiguous().view(-1, e_logits.size(-1))/temperature, hard=False).view(e_logits.size()) for e_logits in listify(inputs)]
    elif method == 'hard_gumbel':
        softmax = [F.gumbel_softmax(e_logits.contiguous().view(-1, e_logits.size(-1))/temperature, hard=True).view(e_logits.size()) for e_logits in listify(inputs)]
    else:
        softmax = [F.softmax(e_logits/temperature, -1) for e_logits in listify(inputs)]
    return [delistify(e) for e in (softmax)]

In [22]:
(edges_hat, nodes_hat) = postprocess((edges_logits, nodes_logits), post_method)   

In [23]:
def get_gen_mols(data, n_hat, e_hat, method):
    (edges_hard, nodes_hard) = postprocess((e_hat, n_hat), method)
    edges_hard, nodes_hard = torch.max(edges_hard, -1)[1], torch.max(nodes_hard, -1)[1]
    mols = [data.matrices2mol(n_.data.cpu().numpy(), e_.data.cpu().numpy(), strict=True) for e_, n_ in zip(edges_hard, nodes_hard)]
    return mols

In [24]:
mols = get_gen_mols(data, nodes_logits, edges_logits, post_method)

RDKit ERROR: [21:11:52] non-ring atom 0 marked aromatic
[21:11:52] non-ring atom 0 marked aromatic
RDKit ERROR: [21:11:52] Explicit valence for atom # 0 C, 5, is greater than permitted
[21:11:52] Explicit valence for atom # 0 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 6 C, 5, is greater than permitted
[21:11:52] Explicit valence for atom # 6 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 8 N, 4, is greater than permitted
[21:11:52] Explicit valence for atom # 8 N, 4, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 5 O, 3, is greater than permitted
[21:11:52] Explicit valence for atom # 5 O, 3, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 0 N, 4, is greater than permitted
[21:11:52] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 4 C, 5, is greater than permitted
[21:11:52] Explicit 

RDKit ERROR: [21:11:52] Explicit valence for atom # 6 N, 4, is greater than permitted
[21:11:52] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 7 C, 5, is greater than permitted
[21:11:52] Explicit valence for atom # 7 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 7 O, 3, is greater than permitted
[21:11:52] Explicit valence for atom # 7 O, 3, is greater than permitted
RDKit ERROR: [21:11:52] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8
[21:11:52] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8

RDKit ERROR: 
RDKit ERROR: [21:11:52] Explicit valence for atom # 6 C, 5, is greater than permitted
[21:11:52] Explicit valence for atom # 6 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] non-ring atom 0 marked aromatic
[21:11:52] non-ring atom 0 marked aromatic
RDKit ERROR: [21:11:52] Explicit valence for atom # 2 C, 5, is greater than permitted
[21:11:52] Explicit valence f

[21:11:52] Explicit valence for atom # 1 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 3 F, 2, is greater than permitted
[21:11:52] Explicit valence for atom # 3 F, 2, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 0 C, 6, is greater than permitted
[21:11:52] Explicit valence for atom # 0 C, 6, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 4 C, 5, is greater than permitted
[21:11:52] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 2 C, 5, is greater than permitted
[21:11:52] Explicit valence for atom # 2 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 0 C, 5, is greater than permitted
[21:11:52] Explicit valence for atom # 0 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] Can't kekulize mol.  Unkekulized atoms: 1 2 4 6 8
[21:11:52] Can't kekulize mol.  Unkekulized atoms: 1 2 4 6

[21:11:52] Can't kekulize mol.  Unkekulized atoms: 0 2 6

RDKit ERROR: [21:11:52] Explicit valence for atom # 0 C, 5, is greater than permitted
[21:11:52] Explicit valence for atom # 0 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] Can't kekulize mol.  Unkekulized atoms: 8
RDKit ERROR: 
[21:11:52] Can't kekulize mol.  Unkekulized atoms: 8

RDKit ERROR: [21:11:52] Explicit valence for atom # 0 C, 5, is greater than permitted
[21:11:52] Explicit valence for atom # 0 C, 5, is greater than permitted
RDKit ERROR: [21:11:52] Explicit valence for atom # 0 O, 4, is greater than permitted
[21:11:52] Explicit valence for atom # 0 O, 4, is greater than permitted
RDKit ERROR: [21:11:52] Can't kekulize mol.  Unkekulized atoms: 1 6 8
[21:11:52] Can't kekulize mol.  Unkekulized atoms: 1 6 8

RDKit ERROR: 
RDKit ERROR: [21:11:52] Explicit valence for atom # 6 O, 3, is greater than permitted
[21:11:52] Explicit valence for atom # 6 O, 3, is greater than permitted
RDKit ERROR: [21:11:52] Explic

RDKit ERROR: [21:11:53] Explicit valence for atom # 0 C, 5, is greater than permitted
[21:11:53] Explicit valence for atom # 0 C, 5, is greater than permitted
RDKit ERROR: [21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 4 6 8
RDKit ERROR: 
[21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 4 6 8

RDKit ERROR: [21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8
RDKit ERROR: 
[21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8

RDKit ERROR: [21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8
[21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8

RDKit ERROR: 
RDKit ERROR: [21:11:53] Explicit valence for atom # 5 O, 4, is greater than permitted
[21:11:53] Explicit valence for atom # 5 O, 4, is greater than permitted
RDKit ERROR: [21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8
[21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8

RDKit ERROR: 
RDKit ERROR: [21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8
[21:

RDKit ERROR: [21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8
RDKit ERROR: 
[21:11:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 8

RDKit ERROR: [21:11:53] non-ring atom 0 marked aromatic
[21:11:53] non-ring atom 0 marked aromatic
RDKit ERROR: [21:11:53] Explicit valence for atom # 0 C, 5, is greater than permitted
[21:11:53] Explicit valence for atom # 0 C, 5, is greater than permitted
RDKit ERROR: [21:11:53] Explicit valence for atom # 3 F, 2, is greater than permitted
[21:11:53] Explicit valence for atom # 3 F, 2, is greater than permitted
RDKit ERROR: [21:11:53] Explicit valence for atom # 0 C, 5, is greater than permitted
[21:11:53] Explicit valence for atom # 0 C, 5, is greater than permitted
RDKit ERROR: [21:11:53] non-ring atom 0 marked aromatic
[21:11:53] non-ring atom 0 marked aromatic
RDKit ERROR: [21:11:53] Explicit valence for atom # 6 N, 4, is greater than permitted
[21:11:53] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit ERROR: [21

# Scores

In [25]:
from utils.utils import *

In [26]:
m0, m1 = all_scores(mols, data, norm=True)

RDKit ERROR: [21:11:54] Explicit valence for atom # 0 O, 3, is greater than permitted
[21:11:54] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [21:11:56] Explicit valence for atom # 0 O, 3, is greater than permitted
[21:11:56] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [21:11:57] Explicit valence for atom # 0 O, 3, is greater than permitted
[21:11:57] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [21:11:57] Explicit valence for atom # 0 O, 3, is greater than permitted
[21:11:57] Explicit valence for atom # 0 O, 3, is greater than permitted


In [27]:
from collections import defaultdict

scores = defaultdict(list)

for k, v in m1.items():
    scores[k].append(v)
for k, v in m0.items():
    scores[k].append(np.array(v)[np.nonzero(v)].mean())

In [28]:
scores

defaultdict(list,
            {'valid': [81.84000253677368],
             'unique': [54.39882697947213],
             'novel': [67.13098729227761],
             'NP': [0.9212796819832025],
             'QED': [0.46756038563467406],
             'Solute': [0.30382607572300896],
             'SA': [0.30612067049105524],
             'diverse': [0.7248585236358851],
             'drugcand': [0.4182255915616082]})

In [29]:
v = MolecularMetrics.valid_filter(mols)

In [30]:
s = set(map(lambda x: Chem.MolToSmiles(x), v))

In [31]:
print(len(s))

2226


In [32]:
from rdkit.Chem import Draw

unique_mols = list(map(lambda x: Chem.MolFromSmiles(x), s))

RDKit ERROR: [21:12:39] Explicit valence for atom # 2 C, 5, is greater than permitted
[21:12:39] Explicit valence for atom # 2 C, 5, is greater than permitted
RDKit ERROR: [21:12:39] Explicit valence for atom # 2 N, 4, is greater than permitted
RDKit ERROR: [21:12:39] Explicit valence for atom # 4 C, 5, is greater than permitted
[21:12:39] Explicit valence for atom # 2 N, 4, is greater than permitted
[21:12:39] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [21:12:39] Explicit valence for atom # 2 C, 5, is greater than permitted
[21:12:39] Explicit valence for atom # 2 C, 5, is greater than permitted


In [33]:
img=Draw.MolsToGridImage(unique_mols[3000:],molsPerRow=8,subImgSize=(200,200), maxMols=500)    

In [34]:
img

In [35]:
df = pd.DataFrame({'SMILES': data} for data in s)

In [36]:
df.to_csv('molgan-'+str(resume_iters)+'.csv', index=False)